In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
import json
from typing import List
import rich

In [3]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

In [4]:
openai = OpenAI()

# Chat Completion API

In [5]:
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    # response_format property is different in chat and responses api
    response_format={
        # schema format is also bit different in chat and responses api
        "type": "json_schema",
        "json_schema": {
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
print(dictionary["name"])

print()
print(response.choices[0].message.parsed) # This will return None, but if we provide pydantic's BaseModel object then we will get parsed object. See example below

{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}
Science Fair

None


In [6]:
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


Note: `response.choices[0].message.parsed` directly returns object

In [7]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    response_format=CalendarEvent
)
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
print(dictionary["name"])

print()
print(response.choices[0].message.parsed)
obj = response.choices[0].message.parsed
print(obj.name)

{"name":"Dinner with Taylor","date":"Tuesday","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor

name='Dinner with Taylor' date='Tuesday' participants=['Leonardo', 'Ivan', 'Alex', 'Taylor']
Dinner with Taylor


# Responses API

In [8]:
response = openai.responses.create(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    # text property is different in chat and responses api
    text={
        # schema format is also bit different in chat and responses api
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.output_text)
dictionary = json.loads(response.output_text)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


Note: In below example we use `openai.responses.parse()` and property for format is `text_format` instead of `text`

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs.py

In [9]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    text_format=CalendarEvent
)
print(response.output_text)
dictionary = json.loads(response.output_text)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


Note: `response.output[0].content[0].parsed` directly returns object

In [10]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    text_format=CalendarEvent
)
print(response.output_text)
print()
print(response)
print()
print(response.output[0].content[0].parsed)
print()
obj = response.output[0].content[0].parsed
print(obj.date)

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}

ParsedResponse[CalendarEvent](id='resp_67d7c52cfe8081928198e754f6051e7a0f5eb5f391530b8f', created_at=1742193964.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ParsedResponseOutputMessage[CalendarEvent](id='msg_67d7c52d58008192a53c8b50182bd69e0f5eb5f391530b8f', content=[ParsedResponseOutputText[CalendarEvent](annotations=[], text='{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}', type='output_text', parsed=CalendarEvent(name='Dinner with Taylor', date='Tuesday night', participants=['Leonardo', 'Ivan', 'Alex', 'Taylor']))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, gen

In [12]:
class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: List[Step]
    final_answer: str

response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathResponse
)

math_reasoning = response.output[0].content[0].parsed
print(math_reasoning)
print()
print(math_reasoning.final_answer)
print()
rich.print(math_reasoning)

steps=[Step(explanation='Start by isolating the term with the variable (8x). You can do this by subtracting 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7 => 8x = -30'), Step(explanation='Next, divide both sides of the equation by 8 to solve for x.', output='8x / 8 = -30 / 8 => x = -30/8'), Step(explanation='Simplify the fraction -30/8 to its simplest form. Both 30 and 8 can be divided by 2.', output='x = -15/4')] final_answer='x = -15/4'

x = -15/4



MathResponse(
    steps=[
        Step(
            explanation='Start by isolating the term with the variable (8x). You can do this by subtracting 7 from 
both sides of the equation.',
            output='8x + 7 - 7 = -23 - 7 => 8x = -30'
        ),
        Step(
            explanation='Next, divide both sides of the equation by 8 to solve for x.',
            output='8x / 8 = -30 / 8 => x = -30/8'
        ),
        Step(
            explanation='Simplify the fraction -30/8 to its simplest form. Both 30 and 8 can be divided by 2.',
            output='x = -15/4'
        )
    ],
    final_answer='x = -15/4'
)